In [1]:
# import langchain
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document

# import vesctor store 

from langchain.vectorstores import chroma
import warnings
warnings.filterwarnings("ignore")

d:\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Documnet loader

loader=DirectoryLoader(
    r"D:\RAG\vector_stor\rag_docs\research",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf-8'}
    
)
documnet=loader.load()
print(f"length of documents {len(documnet)}")
print(f"page content 1 {documnet[0].page_content[:200]}")
print(f"page  content 2 {documnet[1].page_content[:200]}") 
print(f"metadata 1  : {documnet[0].metadata}")

length of documents 2
page content 1  Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. Ther
page  content 2  RAG (Retrieval-Augmented Generation) Architecture:

1. Document Loading: Load documents from various sources
2. Document Splitting: Break documents into smaller chunks
3. Embedding Generation: Conver
metadata 1  : {'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}


#Document splitting

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(
    separators=[",","."," ","\n\n","\n"],
    chunk_size=500,
    chunk_overlap=50,
    length_function=len
)

chunk=splitter.split_documents(documnet)
print(f"length {len(chunk)}")
print(f"page content {chunk[0].page_content[:150]}")
print(f"metadata :{chunk[0].metadata}")

length 6
page content Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from exp
metadata :{'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}


In [28]:
chunk

[Document(metadata={'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}, page_content='Machine Learning Fundamentals\n    \n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models'),
 Document(metadata={'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}, page_content=', while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\n    """,\n    \n    """\n    Deep Learning and Neural Networks\n    \n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of 

#Embedding

In [4]:
from langchain_openai import OpenAIEmbeddings
import os

embed = OpenAIEmbeddings(
    api_key=os.getenv("api_key"),  
    model="text-embedding-3-small",        
    base_url=os.getenv("endpoints")  
)

print(embed)


client=<openai.resources.embeddings.Embeddings object at 0x000001D054F3A3C0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001D054F3ACF0> model='text-embedding-3-small' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base='https://api.euron.one/api/v1/euri' openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [5]:
embed

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001D054F3A3C0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001D054F3ACF0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://api.euron.one/api/v1/euri', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [6]:
sentence="hello world"

In [7]:
vec=embed.embed_query(sentence)
vec

[-0.00676333112642169,
 -0.03919631987810135,
 0.034175805747509,
 0.02876211516559124,
 -0.02478501945734024,
 -0.04203926399350166,
 -0.030289441347122192,
 0.04932808503508568,
 -0.013897152617573738,
 -0.01764741726219654,
 0.015363989397883415,
 -0.027038201689720154,
 -0.020974265411496162,
 -0.027854792773723602,
 0.008619560860097408,
 0.035627517849206924,
 -0.05368323251605034,
 -0.0022720859851688147,
 0.008808586746454239,
 0.04799734428524971,
 0.03710947930812836,
 -0.009247126057744026,
 -0.008778342977166176,
 0.011402016505599022,
 0.014078617095947266,
 -0.0021624513901770115,
 -0.03756314143538475,
 0.04542659968137741,
 0.011250795796513557,
 -0.03964998200535774,
 0.02345428057014942,
 -0.050628580152988434,
 0.012044702656567097,
 -1.5505995179410093e-05,
 0.0160293597728014,
 0.006135766394436359,
 0.03196798637509346,
 0.00336087285540998,
 -0.008604438975453377,
 -0.01055518165230751,
 -0.037381675094366074,
 -0.034508489072322845,
 0.04999345541000366,
 0.0193

In [8]:
persist_dirctory=".chroma.db"

In [9]:
from langchain.vectorstores import Chroma

vectors=Chroma.from_documents(
    documents=chunk,
    embedding=embed,
    persist_directory=persist_dirctory,
    collection_name="RAG_vectors_chroma_db"
)
print(f"vector store created with {vectors._collection.count()}")
print(f"vector store {persist_dirctory}")

vector store created with 12
vector store .chroma.db


#Test Similarity Search from the vectors 

In [10]:
query="what is nlp"

In [11]:
search=vectors.similarity_search(query,k=3)
search

[Document(metadata={'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}, page_content=', natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers \n    excel at sequential data processing.\n    """,\n    \n    """\n    Natural Language Processing (NLP)\n    \n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition'),
 Document(metadata={'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}, page_content=', natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers \n    excel at sequential data processing.\n    """,\n    \n    """\n    Natural 

In [12]:
query="what is machine learning"

In [13]:
search=vectors.similarity_search_with_score(query,k=3)
search

[(Document(metadata={'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}, page_content='Machine Learning Fundamentals\n    \n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models'),
  0.8117268085479736),
 (Document(metadata={'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}, page_content='Machine Learning Fundamentals\n    \n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train

In [ ]:
for i,doc in enumerate (search):
    print(f"\n {i+1}")
    print(f" page content{doc.page_content[:200] } ")
    print(f"metadata {doc.metadata}")


 1
 page contentMachine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There 
metadata {'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}

 2
 page contentMachine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There 
metadata {'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}

 3
 page contentMachine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There 
metadata {'source': 'D:\\RAG\\vector_stor\\rag_docs\\research\\ai_overview.txt'}


In [ ]:
import os, requests
from dotenv import load_dotenv
load_dotenv()
check_endpoint = os.getenv("check_endpoints")
api = os.getenv("api_key") 

response = requests.post(
    check_endpoint,
    headers={"Authorization": f"Bearer {api}"},
    json={
        "model": "gpt-5-nano-2025-08-07", 
        "messages": [{"role": "user", "content": "Hello"}]  
    }
)
 

In [ ]:
response.json()

{'id': 'chatcmpl-CLlvKqSbBgvrIXXYkffgxnLY0l7z0',
 'object': 'chat.completion',
 'created': 1759305142,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi there! How can I help you today? I can answer questions, explain topics, help with writing or coding, plan things, or just chat.\n\nA few examples of what I can do:\n- Explain a concept in simple terms (e.g., quantum physics)\n- Draft or edit emails, resumes, or essays\n- Write or debug code\n- Plan a trip or event\n- Translate or summarize text\n\nTell me what you’re working on, and I’ll dive in.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 7, 'completion_tokens': 491, 'total_tokens': 498}}

In [14]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(
    model="gpt-5-nano-2025-08-07",
    api_key=os.getenv("api_key"),
    openai_api_base= os.getenv("endpoints"))

In [15]:
k=llm.invoke("what is deep learning")

In [16]:
print(k.content)

Deep learning is a subset of machine learning that uses artificial neural networks with many layers to learn from data. The “deep” in deep learning refers to these multiple layers, which allow the model to automatically learn hierarchical representations of information.

Key ideas:
- Representation learning: rather than hand-crafting features, deep learning models learn useful features directly from raw data (images, text, audio, etc.).
- End-to-end learning: a single model can map inputs directly to outputs (e.g., image to label) and train all parts of the network together using gradient-based optimization.
- Backpropagation and gradient descent: learning happens by adjusting weights to reduce a loss function, propagating errors backward through the network.

Common architectures:
- Convolutional Neural Networks (CNNs): excellent for images and spatial data.
- Recurrent Neural Networks (RNNs) and LSTMs/GRUs: good for sequences (time series, language).
- Transformers: highly effective 

#Initialize LLM models,RAG Chin,Promt Template ,Query the System

In [32]:
from langchain.prompts import ChatPromptTemplate

system_prompt='''You are an assistant for question answering.
use the following retrived context to answer the qustion.
if you do not know the answer, you say I do not know.

Context: {context}'''


prompt=ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    ('human','{input}')
])

 

In [33]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are an assistant for question answering.\nuse the following retrived context to answer the qustion.\nif you do not know the answer, you say I do not know.\n\nContext: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain

documents=create_stuff_documents_chain(llm,prompt)

In [35]:
documents

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are an assistant for question answering.\nuse the following retrived context to answer the qustion.\nif you do not know the answer, you say I do not know.\n\nContext: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D0553A8050>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D0553A8830>, r

In [37]:
retriver=vectors.as_retriever(
    search_kwarg={'k':3}
)
retriver

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D054F3BE00>, search_kwargs={})

In [38]:
from langchain.chains import create_retrieval_chain

rag=create_retrieval_chain(retriver,documents)
rag

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D054F3BE00>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are an assistant for question answering.\nuse the following retrived context to answer the qustion.\nif you do not know the answer, 

In [43]:
response=rag.invoke({"input":"what is deep learning and what is machine learning give me diffrence between them?"})

In [47]:
response["answer"]

'Here’s a concise explanation based on the provided context:\n\nWhat is deep learning?\n- Deep learning is a subset of machine learning that is based on artificial neural networks. These networks are inspired by the human brain and consist of layers of interconnected nodes. Deep learning has revolutionized fields like computer vision.\n\nWhat is machine learning?\n- Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. There are three main types: supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models.\n\nDifferences between them:\n- Relationship: Deep learning is a specific method within machine learning; machine learning is the broader field. \n- Techniques: Deep learning relies on deep neural networks with multiple layers; machine learning encompasses a wider range of techniques beyond neural networks.\n- Focus: Deep l